<a href="https://colab.research.google.com/github/GreatLearningAIML1/delhi-may19-batch-sameersuri/blob/master/Stance_Detection_for_the_Fake_News_Challenge_Questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/Datasets/Fake News Challenge/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd

In [0]:
left_df = pd.read_csv("/content/drive/My Drive/Datasets/Fake News Challenge/train_bodies.csv")

In [0]:
right_df = pd.read_csv("/content/drive/My Drive/Datasets/Fake News Challenge/train_stances.csv")

In [0]:
dataset = pd.merge(left_df, right_df, on='Body ID', how='inner')


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [9]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [12]:
#Import libraries

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
t=Tokenizer(num_words=MAX_NB_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True)

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
text_total=['dataset.articleBody','dataset.Headline']

In [0]:
t.fit_on_texts(text_total)

In [16]:
t.word_counts

OrderedDict([('dataset', 2), ('articlebody', 1), ('headline', 1)])

In [17]:
t.word_docs

defaultdict(int, {'articlebody': 1, 'dataset': 2, 'headline': 1})

In [0]:
wrd_idx=t.word_index

In [19]:
t.document_count

2

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
texts = []

In [0]:
articles = []

In [0]:
article_texts = dataset['articleBody'].tolist()

In [0]:
article_sents = []

for article in article_texts:
  article_sents.append(nltk.sent_tokenize(article))

## Check 2:

first element of texts and articles should be as given below. 

In [68]:
article_texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [69]:
article_sents[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from keras.preprocessing.text import text_to_word_sequence

In [0]:
def trim_or_pad(lst, n):
  return lst[:n] + [0]*(n-len(lst))

In [72]:
article_encodings = []

for i in range(len(article_sents)):
  sentence_encodings = []
  n_sents = 0
  for sentence in article_sents[i]:
    # Limit number of sentences to MAX_SENTS
    if(n_sents < MAX_SENTS):
      print("Working on article no. {}, sentence no. {}".format(i, n_sents), end="\r")
      n_sents += 1

      words = text_to_word_sequence(sentence)
      word_encodings = list(itertools.chain.from_iterable(t.texts_to_sequences(words)))
      word_encodings = trim_or_pad(word_encodings, MAX_SENT_LENGTH)
      
      sentence_encodings.append(word_encodings)
    else:
      break
    
  # Handle the case where an article has number of sentences less than MAX_SENTS 
  while(n_sents < MAX_SENTS):
    print("Working on article no. {}, sentence no. {}".format(i, n_sents), end="\r")
    n_sents += 1

    empty_encodings = trim_or_pad([], MAX_SENT_LENGTH)
    sentence_encodings.append(empty_encodings)

  article_encodings.append(sentence_encodings)

In [74]:

article_encodings = np.array(article_encodings)
article_encodings.shape

(49972, 20, 20)

### Check 3:

Accessing first element in data should give something like given below.

In [75]:
article_encodings[0, :, :]


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0,

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [0]:
headline_texts = dataset['Headline'].tolist()


In [0]:
headline_sents = []

for headline in headline_texts:
  headline_sents.append(nltk.sent_tokenize(headline))

In [79]:
headline_encodings = []

for i in range(len(headline_sents)):
  sentence_encodings = []
  n_sents = 0
  for sentence in headline_sents[i]:
    # Limit number of sentences to MAX_SENTS_HEADING
    if(n_sents < MAX_SENTS_HEADING):
      print("Working on headline no. {}, sentence no. {}".format(i, n_sents), end="\r")
      n_sents += 1

      words = text_to_word_sequence(sentence)
      word_encodings = list(itertools.chain.from_iterable(t.texts_to_sequences(words)))
      word_encodings = trim_or_pad(word_encodings, MAX_SENT_LENGTH)
      
      sentence_encodings.append(word_encodings)
    else:
      break
    
  # Handle the case where an headline has number of sentences less than MAX_SENTS_HEADING 
  while(n_sents < MAX_SENTS_HEADING):
    print("Working on headline no. {}, sentence no. {}".format(i, n_sents), end="\r")
    n_sents += 1

    empty_encodings = trim_or_pad([], MAX_SENT_LENGTH)
    sentence_encodings.append(empty_encodings)

  headline_encodings.append(sentence_encodings)


In [80]:
headline_encodings = np.array(headline_encodings)
headline_encodings.shape

(49972, 1, 20)

In [81]:
headline_encodings[0, :, :]


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [83]:
labels = pd.get_dummies(dataset['Stance'])
labels.columns

Index(['agree', 'disagree', 'discuss', 'unrelated'], dtype='object')

In [84]:
labels = labels.values
labels.shape

(49972, 4)

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [85]:

print("Shape of articles tensor : {}".format(article_encodings.shape))
print("Shape of headlines tensor : {}".format(headline_encodings.shape))
print("Shape of labels tensor : {}".format(labels.shape))

Shape of articles tensor : (49972, 20, 20)
Shape of headlines tensor : (49972, 1, 20)
Shape of labels tensor : (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
data = article_encodings.copy

In [0]:
## shuffle the data
article_encodings = article_encodings[indices]

headline_encodings = headline_encodings[indices]

labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:

X_train, X_val, y_train, y_val = train_test_split(article_encodings, labels, test_size=0.2, random_state=10)

In [0]:
X_headline_train, X_headline_val, y_headline_train, y_headline_val = train_test_split(headline_encodings, labels, test_size=0.2, random_state=10)


### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [93]:

X_train = np.reshape(X_train, (X_train.shape[0], MAX_SENTS * MAX_SENT_LENGTH))
print(X_train.shape)

X_val = np.reshape(X_val, (X_val.shape[0], MAX_SENTS * MAX_SENT_LENGTH))
print(X_val.shape)

print(y_train.shape)
print(y_val.shape)

(39977, 400)
(9995, 400)
(39977, 4)
(9995, 4)


In [94]:
X_headline_train = np.reshape(X_headline_train, (X_headline_train.shape[0], MAX_SENTS_HEADING * MAX_SENT_LENGTH))
print(X_headline_train.shape)

X_headline_val = np.reshape(X_headline_val, (X_headline_val.shape[0], MAX_SENTS_HEADING * MAX_SENT_LENGTH))
print(X_headline_val.shape)

print(y_headline_train.shape)
print(y_headline_val.shape)

(39977, 20)
(9995, 20)
(39977, 4)
(9995, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [101]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((MAX_NB_WORDS, 100))


for word, i in t.word_index.items():

	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Bidirectional  
from keras.layers import LSTM
from keras import Input, Model

In [103]:
x_train.shape


(39978, 20, 20)

### Model

In [0]:
from keras.layers import Bidirectional, GlobalMaxPool1D

input_ = Input(shape=(MAX_SENTS * MAX_SENT_LENGTH, ))
x = Embedding(MAX_NB_WORDS, 100, weights=[embedding_matrix])(input_) # embed size is 100 given that we've used glove.6B.100d.txt
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(4, activation="softmax")(x)
model = Model(inputs=input_, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Compile and fit the model

In [106]:
model.fit(X_train, y_train, batch_size=64, epochs=5, validation_data=(X_val, y_val))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 39977 samples, validate on 9995 samples
Epoch 1/5
39977/39977 [==============================] - 787s 20ms/step - loss: 0.3579 - accuracy: 0.8630 - val_loss: 0.3489 - val_accuracy: 0.8671
Epoch 2/5
39977/39977 [==============================] - 792s 20ms/step - loss: 0.3519 - accuracy: 0.8653 - val_loss: 0.3499 - val_accuracy: 0.8671
Epoch 3/5
39977/39977 [==============================] - 793s 20ms/step - loss: 0.3518 - accuracy: 0.8653 - val_loss: 0.3530 - val_accuracy: 0.8671
Epoch 4/5
39977/39977 [==============================] - 793s 20ms/step - loss: 0.3515 - accuracy: 0.8653 - val_loss: 0.3497 - val_accuracy: 0.8671
Epoch 5/5
39977/39977 [==============================] - 790s 20ms/step - loss: 0.3512 - accuracy: 0.8653 - val_loss: 0.3491 - val_accuracy: 0.8671


## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)